In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

account = pd.read_csv("data/account.csv")
transaction = pd.read_csv("data/transaction.csv")
loan = pd.read_csv("data/loan.csv")
card = pd.read_csv("data/card.csv")
client = pd.read_csv("data/client.csv")
disp = pd.read_csv("data/disp.csv")
order = pd.read_csv("data/order.csv")


/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [9]:
n= 3

display(account.head(n))
display(transaction.head(n))
display(loan.head(n))

display(card.head(n))
display(client.head(n))
display(disp.head(n))
display(order.head(n))

,account_id,district_id,frequency,date
0,576,55,POPLATEK MESICNE,930101
1,3818,74,POPLATEK MESICNE,930101
2,704,55,POPLATEK MESICNE,930101


,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,930101,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN
1,171812,576,930101,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN
2,207264,704,930101,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN


,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,930705,96396,12,8033.0,B
1,5316,1801,930711,165960,36,4610.0,A
2,6863,9188,930728,127080,60,2118.0,A


,card_id,disp_id,type,issued
0,1005,9285,classic,931107 00:00:00
1,104,588,classic,940119 00:00:00
2,747,4915,classic,940205 00:00:00


,client_id,birth_number,district_id
0,1,706213,18
1,2,450204,1
2,3,406009,1


,disp_id,client_id,account_id,type
0,1,1,1,OWNER
1,2,2,2,OWNER
2,3,3,2,DISPONENT


,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO


## client info

In [13]:
client_info =  pd.merge(account,disp,on='account_id')
client_info.head(2)


,account_id,district_id,frequency,date,disp_id,client_id,type
0,576,55,POPLATEK MESICNE,930101,692,692,OWNER
1,576,55,POPLATEK MESICNE,930101,693,693,DISPONENT


## transactions

In [82]:
transaction["date"] =  pd.to_datetime(transaction["date"],format="%y%m%d")
transaction.head(3)

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,1993-01-01,PRIJEM,VKLAD,700.0,700.0,NaN,NaN,NaN
1,171812,576,1993-01-01,PRIJEM,VKLAD,900.0,900.0,NaN,NaN,NaN
2,207264,704,1993-01-01,PRIJEM,VKLAD,1000.0,1000.0,NaN,NaN,NaN


# /GetCreditRiskAttributes
calculating and merging credit risk attributes in pandas. to be inserted into new postgres tables

## Loan payments and credits


In [144]:
#1 sum_loan_payments
loan_payments = transaction[(transaction["k_symbol"]=="UVER")]
sum_loan_payments = loan_payments.groupby(["account_id"])[["amount"]].sum()
sum_loan_payments.head(3)


#2 average_monthly_loan_payments_complex
average_monthly_loan_payments_complex = loan_payments.groupby([ "account_id"])[["amount"]].mean()


#3 sum_total_credits_monthly"
credits = transaction[(transaction["type"]=="PRIJEM")]
credits_1998 = credits[credits["date"].dt.year == 1998]
sum_total_credits_monthly_1998 = credits_1998.groupby([ "account_id",
                                                    pd.Grouper(key='date',freq="M")])[["amount"]].sum()

sum_total_credits_monthly_1998.reset_index(1,inplace=True)
sum_total_credits_monthly_1998["date"] = sum_total_credits_monthly_1998["date"].dt.month
sum_total_credits_monthly_1998.columns = ["month","amount"]


#4 sum_total_credits_n_months_ago
sum_total_credits_n_month_ago = sum_total_credits_monthly_1998.reset_index()\
                                                              .pivot(columns='month', index='account_id', values='amount')
sum_total_credits_n_month_ago.head(3)


#5 balance_min
balance_min = loan_payments.groupby([ "account_id"])[["balance"]].min()


#6 average_monthly_debit
debits = transaction[(transaction["type"]=="VYDAJ")]
debits_1998 = credits[credits["date"].dt.year == 1998]
average_monthly_debit_1998 = debits_1998.groupby([ "account_id"])[["amount"]].mean()






,amount
account_id,
1,1816.612000
2,10316.626923
3,3591.228125
4,2824.195833
5,2563.920833
...,...
11333,5459.077778
11349,22178.156000
11359,14277.416000


In [121]:
credits[credits["account_id"]==576]["operation"].value_counts()

PREVOD Z UCTU    72
VKLAD             1
Name: operation, dtype: int64